# Data Mining Lab Week 5: Data Warehousing and On-Line Analytical Processing (OLAP)

## Introduction 

The aim of this lab is for students to get experience with **Data Warehousing** and **On-line Analytical Processing (OLAP)** covered in week 5, and more specifically with the concepts of **data cubes**, **data cube measures**, **typical OLAP operations**, and **data cube computation**.

- This lab is the first part of a **two-week assignment** that covers weeks 5 and 6.
- This lab corresponds to the **second assignment**, accounting for 10% of your overall grade. Questions in this lab sheet will contribute to 5% of your overall grade; questions in the lab sheet for week 6 will cover for another 5% of your overall grade.
- <font color = 'maroon'>The last section of this notebook includes the questions that are assessed towards your final grade.</font> 

## Important notes about the assignment: 

- **PLAGIARISM** <ins>is an irreversible non-negotiable failure in the course</ins> (if in doubt of what constitutes plagiarism, ask!). 
- The total assessed coursework is worth 40% of your final grade.
- There will be 9 lab sessions and 4 assignments.
- One assignment will cover 2 consecutive lab sessions and will be worth 10 marks (percentages of your final grade).
- The submission cut-off date will be 7 days after the deadline and penalties will be applied for late submissions in accordance with the School policy on late submissions.
- You are asked to submit a **report** that should answer the questions specified in the last section of this notebook. The report should be in **PDF format** (so **NOT** *doc, docx, notebook* etc). It should be well identified with your name, student number, assignment number (for instance, Assignment 1), module, and marked with question numbers. 
- No other means of submission other than submitting your assignment through the appropriate QM+ link are acceptable at any time. Submissions sent via email will **not** be considered.
- Please name your report as follows: Assignment1-StudentName-StudentNumber.pdf
- Cases of **Extenuating Circumstances (ECs)** have to go through the proper procedure of the School in due time. Only cases approved by the School in due time can be considered.

## 1. Introduction to Cubes

This chapter describes step-by-step how to use Cubes (http://cubes.databrewery.org/), a lightweight Python framework and set of tools for development of reporting and analytical applications, Online Analytical Processing (OLAP), multidimensional analysis and browsing of aggregated data. We will be working with v1.1 of Cubes. Cubes features:
- a logical view of analysed data - how analysts look at data, how they think of data, not not how the data are physically implemented in the data stores
- OLAP and aggregated browsing (default backend is for relational databse - ROLAP)
- hierarchical dimensions (attributes that have hierarchical dependencies, such as category-subcategory or country-region)
- multiple hierarchies in a dimension
- localizable metadata and data (see Localization)
- authentication and authorization of cubes and their data
- pluggable data warehouse – plug-in other cube-like (multidimensional) data sources

Cubes is meant to be used by application builders that want to provide analytical functionality. Cubes also relies on methods from SQLAlchemy (https://www.sqlalchemy.org/), an open-source SQL toolkit and object-relational mapper for Python.

## 2. Data Preparation

The example data used here are International Bank for Reconstruction and Development (IBRD) Balance Sheet. For this example we will be using the CSV file "IBRD_Balance_Sheet__FY2010.csv" which is provided in the supplementary material for the lab. The CSV file includes records which are characterised by a Category (and subcategories), Line Item, Fiscal Year, and Amount (in US$ millions). We first start with imports:

In [ ]:
# If you are using Google Colab, you would need to run the below line to install Cubes. 
# The below line might not be needed if you are running a local python installation with Cubes installed.
!pip install cubes

from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv

     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 92kB 6.5MB/s 
  Created wheel for cubes: filename=cubes-1.1-cp36-none-any.whl size=151591 sha256=2ca9b733147c1d7817ca27e8a2f7fc37ab0e083af9da52d736f40f561a3844a0
  Stored in directory: /root/.cache/pip/wheels/1f/89/09/c361b647a6cc65335af4bc1f27b07abd123727f429819d2590
  Created wheel for expressions: filename=expressions-0.2.3-cp36-none-any.whl size=6445 sha256=f79270aadd799b965199d3c178be99a20897ccd8e81addba3733b8b57b03d460
  Stored in directory: /root/.cache/pip/wheels/56/67/97/1a31fbf02237daec61a3d75439d782cf13dfc4af31e8b4e80e
Successfully built cubes expressions


We can now load the data, create a table and populate it with contents of the CSV file:

In [ ]:
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "IBRD_Balance_Sheet__FY2010.csv",
                      table_name="ibrd_balance",
                      fields=[
                          ("category", "string"),
                          ("category_label", "string"),
                          ("subcategory", "string"),
                          ("subcategory_label", "string"),
                          ("line_item", "string"),
                          ("year", "integer"),
                          ("amount", "integer")],
                      create_id=True
                     )

## 3. Creating a data cube

Everything in Cubes happens in an *analytical workspace*. It contains cubes, maintains connections to the data stores (with cube data), provides connection to external cubes and more.

The workspace properties are specified in a configuration file slicer.ini (default name). The first thing we have to do is to specify a data store – a database which will host the cube’s data:

In [ ]:
from cubes import Workspace

workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")

The structure of data cubes (in terms of dimensions, measures, and aggregates) is specified in JSON files. We now import file 'tutorial_model.json' (found in the lab supplementary material) which includes an example model of the data cube, dimension tables, and aggregate functions for the CSV file we loaded previously.

In [ ]:
workspace.import_model("tutorial_model.json")

**Please make sure to inspect the structure of the above JSON file - this will be relevant for one of the assignment questions.**

We can now create a data cube based on the above data cube model and data table:

In [ ]:
cube = workspace.cube("ibrd_balance")

## 4. Aggregations and OLAP operations

*Browser* is an object that does the actual aggregations and other data queries for a cube. To obtain one:

In [ ]:
browser = workspace.browser(cube)

We can now compute aggregates of the data cube as specified by the data cube model. For computing the total count of records:

In [ ]:
result = browser.aggregate()
result.summary["record_count"]

62

For computing a sum of the amount:

In [ ]:
result.summary["amount_sum"]

1116860

Now we can try to compute aggregates by year:

In [ ]:
result = browser.aggregate(drilldown=["year"])
for record in result:
    print(record)

{'year': 2009, 'amount_sum': 550840, 'record_count': 31, 'amount_min': -1683, 'amount_max': 110040}
{'year': 2010, 'amount_sum': 566020, 'record_count': 31, 'amount_min': -3043, 'amount_max': 128577}


Or compute aggregates by item category:

In [ ]:
result = browser.aggregate(drilldown=["item"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'amount_sum': 558430, 'record_count': 32}
{'item.category': 'e', 'item.category_label': 'Equity', 'amount_sum': 77592, 'record_count': 8}
{'item.category': 'l', 'item.category_label': 'Liabilities', 'amount_sum': 480838, 'record_count': 22}


We can also perform *slicing* and *dicing* operations on the data cube. The below example performs a slicing operation on the data cube by selecting only entries with the year being 2009, and displays aggregates according to the item category. Here, a *cell* defines a point of interest – portion of the cube to be aggergated or browsed.

In [ ]:
import cubes as cubes
cuts = [cubes.PointCut("year", ["2009"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["item"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'amount_sum': 275420, 'record_count': 16}
{'item.category': 'e', 'item.category_label': 'Equity', 'amount_sum': 40037, 'record_count': 4}
{'item.category': 'l', 'item.category_label': 'Liabilities', 'amount_sum': 235383, 'record_count': 11}


It's worth noting that in Cubes, slicing operations can be created by either specifying a "point cut" which selects a single value of an attribute in a given dimension (called using the cubes.PointCut() function as above), or by specifying a "range cut", which selects a range of values for a given dimension. The range cut can be called using the cubes.RangeCut() function, which takes as input the attribute name, the minimum value of the specified range, and the maximum value of the range.

Similarly, we can perform a *dicing* operation on the data cube by performing a selection on two or more dimensions. The below example performs a dicing operation on the data cube, selecting entries with the year being 2009 and the item category being "a", and displays the aggregate results:

In [ ]:
cuts = [cubes.PointCut("year", ["2009"]),cubes.PointCut("item", ["a"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell,drilldown=["item"])
result.summary

{'amount_sum': 275420, 'record_count': 16}

We can also *drill down* lower in the Category hierarchy. Here, we perform a dicing operation to select records with year being 2009 and item category being "a" (corresponding to assets), and show aggregates for each subcategory level.

In [ ]:
cuts = [cubes.PointCut("year", ["2009"]),cubes.PointCut("item", ["a"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell,drilldown=["item:subcategory"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'da', 'item.subcategory_label': 'Derivative Assets', 'amount_sum': 123065, 'record_count': 4}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'dfb', 'item.subcategory_label': 'Due from Banks', 'amount_sum': 3044, 'record_count': 2}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'i', 'item.subcategory_label': 'Investments', 'amount_sum': 41012, 'record_count': 1}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'lo', 'item.subcategory_label': 'Loans Outstanding', 'amount_sum': 103657, 'record_count': 1}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'nn', 'item.subcategory_label': 'Nonnegotiable', 'amount_sum': 1202, 'record_count': 1}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'oa', 'item.subcategory_label': 'Other Assets', 'amount_sum': 2247, 'record_count': 3}
{'ite

## <font color = 'maroon'>Assignment 2 (part 1/2)</font>

1. A data warehouse for a music streaming company consists of the dimensions song, user, time (time and date of when the user listened to a song), and the two measures count (how many times a user listened to the song) and fee (fee paid by the streaming company to the artist every time a user listens to that song). 
  1. Draw a schema diagram for the above data warehouse using either a star, snowflake, or fact constellation schema. [0.5 marks out of 5]
  2. Starting with the base cuboid [time, user, song], what specific OLAP operations should be performed in order to list the total fee collected for a given song for a given month of a given year (e.g. October 2020)? [0.5 marks out of 5]
  3. Assume that the time dimension has 5 levels (e.g. day, day of week, month, quarter, year); and that the song and user dimensions both have 1 level (not including the virtual level 'all'). How many cuboids will this cube contain (including the base and apex cuboids)? [0.5 marks out of 5]
  

2. Suppose we have access to a data cube that contains information on rainfall for specific regions; the data cube has dimensions region, precipitation, and time.
  1. Assuming that we would like to compute the total amount of rainfall for a given region and month, which *data cube measure* would we use? To which category of data cube measures does this particular measure fall into? [0.25 marks out of 5]
  2. Assuming that we would like to compute the average rainfall for a given region and month, which *data cube measure* would we use? To which category of data cube measures does this particular measure fall into? [0.25 marks out of 5]
  

3. Suppose that a car rental company has a data warehouse that holds record ID lists of vehicles in terms of brands (Audi, Ford, Mini) and store branches (Tower Hamlets, Newham, Hackney). Each record consists of a combination of vehicle brand and branch. We would like to index the OLAP data using bitmap indices. Produce the *base table* for record IDs, and the corresponding *bitmap index tables* for vehicle brand and store branch. [1 mark out of 5]
  

4. Using the same CSV file and data cube in the above lab tutorial, modify the "tutorial_model.json" file to include aggregate measures for the minimum and maximum amount in the data cube. Using these implemented aggregate measures, produce the values for the minimum and maximum amount in the data per year. Make sure to show your workings in the PDF report. [0.5 marks out of 5]


5. Using the CSV file "country-income.csv" (found in the supplementary lab documents), perform the following:
  1. Load the CSV file using Cubes, create a JSON file for the data cube model, and create a data cube for the data. Use as dimensions the region, age, and online shopper fields. Use as measure the income. Define aggregate functions in the data cube model for the total, average, minimum, and maximum income. In your PDF report, show the relevant scripts and files created. [0.5 marks out of 5]
  2. Using the created data cube and data cube model, produce aggregate results for: the whole data cube; results per region; results per online shopping activity; and results for all people aged between 40 and 50. [1 mark out of 5]
  

# Q5

#### 5.1

In [1]:

#Loading the data:
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "country_income.csv",
                      table_name="country_income",
                      fields=[
                          ("region", "string"),
                          ("age", "integer"),
                          ("income", "integer"),
                          ("online_shopper", "string")],
                      create_id=True
                     )

NameError: ignored

In [ ]:
from cubes import Workspace

workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")

In [ ]:
workspace.import_model("country_income.json")

In [ ]:
cube = workspace.cube("country_income")

In [ ]:
browser = workspace.browser(cube)
result = browser.aggregate()

#### 5.2

In [ ]:
#Results from whole data-cube
print('Total sum:', result.summary["income_sum"])
print('Average income:', result.summary["income_avg"])
print('Smallest income record:', result.summary["income_min"])
print('Largest income record:', result.summary["income_max"])

Total sum: 768200
Average income: 76820.0
Smallest income record: 57600
Largest income record: 99600


In [ ]:
#Per region
result = browser.aggregate(drilldown=["region"])
for record in result:
    print(record)

{'region': 'Brazil', 'income_sum': 193200, 'income_avg': 64400.0, 'income_min': 57600, 'income_max': 73200}
{'region': 'India', 'income_sum': 331200, 'income_avg': 82800.0, 'income_min': 69600, 'income_max': 94800}
{'region': 'USA', 'income_sum': 243800, 'income_avg': 81266.66666666667, 'income_min': 64800, 'income_max': 99600}


In [ ]:
#Per online shopping activity
result = browser.aggregate(drilldown=["online_shopper"])
for record in result:
    print(record)

{'online_shopper': 'No', 'income_sum': 386400, 'income_avg': 77280.0, 'income_min': 62400, 'income_max': 99600}
{'online_shopper': 'Yes', 'income_sum': 381800, 'income_avg': 76360.0, 'income_min': 57600, 'income_max': 94800}


In [ ]:
#For shoppers between age 40-50
import cubes as cubes
import numpy as np
cuts = [cubes.RangeCut("age", [40], [50])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell)

print('Total sum:', result.summary["income_sum"])
print('Average income:', np.round(result.summary["income_avg"], 2))
print('Lowest income:', result.summary["income_min"])
print('Highest income:', result.summary["income_max"])

Total sum: 451400
Average income: 75233.33
Lowest income: 62400
Highest income: 86400
